<a href="https://colab.research.google.com/github/2IS239-Data-Analytics/Code_along_2/blob/main/Scale%2C_Standardize%2C_or_Normalize_with_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial Scale, Standardize, or Normalize with scikit-learn
### När ska man använda MinMaxScaler, RobustScaler, StandardScaler, och Normalizer
### Attribution: Jeff Hale

### Varför är det ofta nödvändigt att genomföra så kallad variable transformation/feature scaling det vill säga, standardisera, normalisera eller på andra sätt ändra skalan på data vid dataaalys?

Som jag gått igenom på föreläsning 3 kan data behöva formateras (variable transformation) för att förbättra prestandan hos många algoritmer för dataanalys. En typ av formatering av data, som går att göra på många olika sätt, är så kallad skalning av attribut (feature scaling). Det kan finnas flera anledningar till att data kan behöv skalas, några exempel är:

* Exempelvis neurala nätverk, regressionsalgoritmer och K-nearest neighbors fungerar inte lika bra om inte de attribut (features) som algoritmen använder befinner sig i relativt lika skalor. 

* Vissa av metoderna för att skala, standardisera och normalisera kan också minska den negativa påverkan outliers kan ha i vissa algoritmer.

* Ibland är det också av vikt att ha data som är normalfördelat (standardiserat) 

*Med skala menas inte den skala som hänsyftas på exempelvis kartor där det brukar anges att skalan är 1:50 000 vilket tolkas som att varje avstånd på kartan är 50 000 ggr kortare än i verkligheten.* 


In [ ]:
#Importerar de bibliotek vi behöver
import numpy as np 
import pandas as pd 
from sklearn import preprocessing

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
#Denna kod sätter upp hur matplotlib ska visa grafer och plotar
%matplotlib inline
matplotlib.style.use('ggplot')

#Generera lite input 
#(den som är extremt intresserad kan läsa följande, intressanta och roliga förklaring kring varför random.seed egentligen är pseudorandom)
#https://www.sharpsightlabs.com/blog/numpy-random-seed/
np.random.seed(34)

# Original Distributions 

Data som det kan se ut i original, alltså när det samlats in, innan någon pre-processing har genomförts.

För att ha data att använda i övningarna skapar nedanstående kod ett antal randomiserade spridningar av data

In [ ]:
#skapa kolumner med olika fördelningar 
df = pd.DataFrame({ 
    'beta': np.random.beta(5, 1, 1000) * 60,        # beta
    'exponential': np.random.exponential(10, 1000), # exponential
    'normal_p': np.random.normal(10, 2, 1000),      # normal platykurtic
    'normal_l': np.random.normal(10, 10, 1000),     # normal leptokurtic
})

# make bimodal distribution
first_half = np.random.normal(20, 3, 500) 
second_half = np.random.normal(-20, 3, 500) 
bimodal = np.concatenate([first_half, second_half])

df['bimodal'] = bimodal

# create list of column names to use later
col_names = list(df.columns)

## Uppgift 1: 

a. Plotta de kurvor som skapats i ovanstående cell i en och samma koordinatsystem med hjälp av [seaborn biblioteket](https://seaborn.pydata.org/api.html#distribution-api).

>Se till att det är tydligt vilken kurva som representerar vilken distribution.
>
>Koden för själva koordinatsystemet är given, fortsätt koda i samma cell
>
>HINT! alla fem är distribution plots

In [ ]:
# plot original distribution plot
fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('Original Distributions')

#De fem kurvorna
sns.kdeplot(df['beta'], ax=ax1)
sns.kdeplot(df['exponential'], ax=ax1)
sns.kdeplot(df['normal_p'], ax=ax1)
sns.kdeplot(df['normal_l'], ax=ax1)
sns.kdeplot(df['bimodal'], ax=ax1);

b. Visa de fem första raderna i den dataframe som innehåller alla distributioner.

In [ ]:
df.head()

c. För samtliga fem attribut, beräkna:

* medel
* median

Vad för bra metod kan användas för att få ett antal statistiska mått på en dataframe? Hämta denna information med denna metod.

In [ ]:
df.describe()

In [ ]:
df.skew()

d. I pandas kan du plotta din dataframe på några olika sätt. Gör en plot för att ta reda på hur skalan på de olika attibuten ser ut, befinner sig alla fem i ungefär samma skala?


In [ ]:
df.plot()

* Samtliga värden ligger inom liknande intervall

e. Vad händer om följande kolumn med randomiserade värden läggs till?

In [ ]:
new_column = np.random.normal(1000000, 10000, (1000,1))
df['new_column'] = new_column
col_names.append('new_column')
df['new_column'].plot(kind='kde')

In [ ]:
# plot våra originalvärden tillsammans med det nya värdet
fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('Original Distributions')

sns.kdeplot(df['beta'], ax=ax1)
sns.kdeplot(df['exponential'], ax=ax1)
sns.kdeplot(df['normal_p'], ax=ax1)
sns.kdeplot(df['normal_l'], ax=ax1)
sns.kdeplot(df['bimodal'], ax=ax1);
sns.kdeplot(df['new_column'], ax=ax1);

Hur gick det?

Testar några olika sätt att skala dataframes..

### MinMaxScaler

MinMaxScaler subtraherar varje värde i en kolumn med medelvärdet av den kolumnen och dividerar sedan med antalet värden. 

In [ ]:
mm_scaler = preprocessing.MinMaxScaler()
df_mm = mm_scaler.fit_transform(df)

df_mm = pd.DataFrame(df_mm, columns=col_names)

fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('After MinMaxScaler')

sns.kdeplot(df_mm['beta'], ax=ax1)
sns.kdeplot(df_mm['exponential'], ax=ax1)
sns.kdeplot(df_mm['normal_p'], ax=ax1)
sns.kdeplot(df_mm['normal_l'], ax=ax1)
sns.kdeplot(df_mm['bimodal'], ax=ax1)
sns.kdeplot(df_mm['new_column'], ax=ax1);

Vad har hänt med värdena?

In [ ]:
df_mm['beta'].min()

In [ ]:
df_mm['beta'].max()

Vi jämför med min och maxvärde för varje kolumn innan vi normaliserade vår dataframe

In [ ]:
mins = [df[col].min() for col in df.columns]
mins

In [ ]:
maxs = [df[col].max() for col in df.columns]
maxs

Kollar min och max för alla kolumner efter skalning.

In [ ]:
mins = [df_mm[col].min() for col in df_mm.columns]
mins

In [ ]:
maxs = [df_mm[col].max() for col in df_mm.columns]
maxs

Vad har hänt?

### RobustScaler

RobustScaler subtraherar med medianen för kolumnen och dividerar med kvartilavståndet (skillnaden mellan största 25% och minsta 25%) 

In [ ]:
r_scaler = preprocessing.RobustScaler()
df_r = r_scaler.fit_transform(df)

df_r = pd.DataFrame(df_r, columns=col_names)

fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('After RobustScaler')

sns.kdeplot(df_r['beta'], ax=ax1)
sns.kdeplot(df_r['exponential'], ax=ax1)
sns.kdeplot(df_r['normal_p'], ax=ax1)
sns.kdeplot(df_r['normal_l'], ax=ax1)
sns.kdeplot(df_r['bimodal'], ax=ax1)
sns.kdeplot(df_r['new_column'], ax=ax1);

Vi kollar igen min och max efteråt

In [ ]:
mins = [df_r[col].min() for col in df_r.columns]
mins

In [ ]:
maxs = [df_r[col].max() for col in df_r.columns]
maxs

Vad har hänt?

### StandardScaler

StandardScaler skalar varje kolumn till att ha 0 som medelvärde och standardavvikelsen 1 

In [ ]:
s_scaler = preprocessing.StandardScaler()
df_s = s_scaler.fit_transform(df)

df_s = pd.DataFrame(df_s, columns=col_names)

fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('After StandardScaler')

sns.kdeplot(df_s['beta'], ax=ax1)
sns.kdeplot(df_s['exponential'], ax=ax1)
sns.kdeplot(df_s['normal_p'], ax=ax1)
sns.kdeplot(df_s['normal_l'], ax=ax1)
sns.kdeplot(df_s['bimodal'], ax=ax1)
sns.kdeplot(df_s['new_column'], ax=ax1);

Vi kontrollerar min och max återigen

In [ ]:
mins = [df_s[col].min() for col in df_s.columns]
mins

In [ ]:
maxs = [df_s[col].max() for col in df_s.columns]
maxs

Vad har hänt? I jämförelse med de två innan?

# Normalizer

Normaliser transformerar rader istället för kolumner genom att (default) beräkna den Euclidiska normen som är roten ur summan av roten ur samtliga värden. Kallas för l2.

In [ ]:
n_scaler = preprocessing.Normalizer()
df_n = n_scaler.fit_transform(df)

df_n = pd.DataFrame(df_n, columns=col_names)

fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('After Normalizer')

sns.kdeplot(df_n['beta'], ax=ax1)
sns.kdeplot(df_n['exponential'], ax=ax1)
sns.kdeplot(df_n['normal_p'], ax=ax1)
sns.kdeplot(df_n['normal_l'], ax=ax1)
sns.kdeplot(df_n['bimodal'], ax=ax1)
sns.kdeplot(df_n['new_column'], ax=ax1);

Min och max efter skalning

In [ ]:
mins = [df_n[col].min() for col in df_n.columns]
mins

In [ ]:
maxs = [df_n[col].max() for col in df_n.columns]
maxs

Vad har hänt?

Nu tar vi en titt på alla olika sätt att skala tillsammans, dock skippar vi normalizern då det är väldigt ovanligt att man vill skala om rader.

### Kombinerad plot

In [ ]:
#Själva figuren
fig, (ax0, ax1, ax2, ax3) = plt.subplots(ncols=4, figsize=(20, 8))


ax0.set_title('Original Distributions')

sns.kdeplot(df['beta'], ax=ax0)
sns.kdeplot(df['exponential'], ax=ax0)
sns.kdeplot(df['normal_p'], ax=ax0)
sns.kdeplot(df['normal_l'], ax=ax0)
sns.kdeplot(df['bimodal'], ax=ax0)
sns.kdeplot(df['new_column'], ax=ax0);


ax1.set_title('After MinMaxScaler')

sns.kdeplot(df_mm['beta'], ax=ax1)
sns.kdeplot(df_mm['exponential'], ax=ax1)
sns.kdeplot(df_mm['normal_p'], ax=ax1)
sns.kdeplot(df_mm['normal_l'], ax=ax1)
sns.kdeplot(df_mm['bimodal'], ax=ax1)
sns.kdeplot(df_mm['new_column'], ax=ax1);


ax2.set_title('After RobustScaler')

sns.kdeplot(df_r['beta'], ax=ax2)
sns.kdeplot(df_r['exponential'], ax=ax2)
sns.kdeplot(df_r['normal_p'], ax=ax2)
sns.kdeplot(df_r['normal_l'], ax=ax2)
sns.kdeplot(df_r['bimodal'], ax=ax2)
sns.kdeplot(df_r['new_column'], ax=ax2);


ax3.set_title('After StandardScaler')

sns.kdeplot(df_s['beta'], ax=ax3)
sns.kdeplot(df_s['exponential'], ax=ax3)
sns.kdeplot(df_s['normal_p'], ax=ax3)
sns.kdeplot(df_s['normal_l'], ax=ax3)
sns.kdeplot(df_s['bimodal'], ax=ax3)
sns.kdeplot(df_s['new_column'], ax=ax3);

Efter samtliga transformationer är värdena på en mer lika skala. MinMax hade varit att föredra här eftersom den ger minst förskjutning av värdena i förhållande till varandra. Det är samma avstånd som i originalet, de andra två skalningsmetoderna ändrar avstånden mellan värdena vilket kommer påverka modellens korrekthet. 